<a href="https://colab.research.google.com/github/BlackArctic20/BluepringComparisonIA/blob/main/Eduardo1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pdf2image


In [2]:
!apt-get install -y poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.6).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [10]:
# -*- coding: utf-8 -*-
"""
Created on Tue Mar 25 10:59:43 2025

@author: HUJ1GA
"""

import tensorflow as tf
import numpy as np
from pdf2image import convert_from_path
import cv2
from PIL import Image
import os

# Function to convert the first page of a PDF to a PNG image
def pdf_to_image(pdf_path, output_path):
    """Converts the first page of a PDF to a PNG image."""
    try:
        pages = convert_from_path(pdf_path, dpi=200)  # Convert PDF to list of PIL Image objects
    except Exception as e:
        print(f"Error converting PDF: {e}")
        return None

    if not os.path.exists(output_path):
        os.makedirs(output_path)

    if pages:
        image_path = os.path.join(output_path, 'page.png')
        pages[0].save(image_path, 'PNG')
        return image_path
    else:
        print("No pages converted from PDF.")
        return None

# Function to preprocess image for TensorFlow
def preprocess_image(image_path):
    """Preprocesses an image for TensorFlow."""
    if not image_path:
        return None

    img = cv2.imread(image_path)
    if img is None:
        print("Error loading image.")
        return None

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB
    img = cv2.resize(img, (512, 512))  # Resize for consistency
    img_tensor = tf.convert_to_tensor(img, dtype=tf.float32) / 255.0  # Normalize
    return img_tensor

# Function to compare images and highlight differences
def compare_images(old_img_tensor, new_img_tensor, output_path):  # Corrected function
    """Compares two images and highlights differences in red."""
    if old_img_tensor is None or new_img_tensor is None:
        print("Cannot compare. One or both images are invalid")
        return

    # Compute absolute difference
    diff = tf.abs(old_img_tensor - new_img_tensor)

    # Threshold to highlight significant differences
    threshold = 0.1  # Adjust as needed
    diff_binary = tf.where(diff > threshold, 1.0, 0.0)
    diff_binary_np = diff_binary.numpy()

    # Create overlay with correct dimensions
    overlay = np.zeros_like(old_img_tensor.numpy())

     # Apply red color using multi-dimensional slicing
    red_indices = diff_binary_np > 0

    # Ensure red_indices is 2D (important for single-channel images)
    if red_indices.ndim == 3:
        red_indices = np.any(red_indices, axis=-1)

    overlay[red_indices] = [255, 0, 0]  # Set differing pixels to red using correct type


    # Combine original image with overlay
    blended_image = old_img_tensor.numpy() * 255 + overlay
    blended_image = np.clip(blended_image, 0, 255).astype(np.uint8) # Ensure appropriate data type


    cv2.imwrite(output_path, cv2.cvtColor(blended_image, cv2.COLOR_RGB2BGR)) #Convert back to BGR before saving.
    print(f"Differences saved to {output_path}")




# Main execution
def main():
    # ... (rest of the main function is the same)
    pass # Added a pass statement as a placeholder. Replace with your main logic.

if __name__ == "__main__":
    main() # Call the main function when the script is executed.
